In [50]:
from random import randrange, choice
import numpy as np
locs = {
    'agent_x':0,
    'agent_z':0,
    'agent_rot':0,
    'goals':"",
    'wall_x':0,
    'wall_z':0,
    'wall_size':0,
    'wall_tall':0
}
def valid_loc(diff=0, num_goals=3,side='left'):
    res = locs.copy()
    agent_x = randrange(5,35)
    agent_z = randrange(1,10-diff)
    wall_tall = randrange(2,5)
    wall_x = agent_x + choice([1,-1])*randrange(diff*4,1+diff*8)
    wall_x = max(min(wall_x,35-diff*2),5+diff*2)
    wall_z = agent_z + randrange(6+diff,7+diff*4)
    if wall_z>39:
        wall_z = 35
    wall_size = randrange(2+diff*3,3+diff*5)
    if (wall_x + wall_size/2) > 35:
        wall_size = 39-wall_x
    if (wall_x-wall_size/2)<5:
        wall_size = int(wall_x/2-1)
        if wall_size<2:
            wall_size=2
    goals = gen_goals(wall_x, wall_z,wall_size, diff, num_goals, side)
    wall_diff = wall_x - agent_x
    if wall_diff>0: # wall is to the right
        agent_rot = int(60*wall_diff/40)
    else: # wall is to the left
        agent_rot = int(360+(60*wall_diff/40))
        
    for k in locs:
        locs[k] = locals()[k]
    return locs
a_goal = """
        - !Item
          name: GoodGoal
          positions:
          - !Vector3 {{x: {x}, y: 0, z: {z}}}
          sizes:
          - !Vector3 {{x: 1, y: 1, z: 1}}"""
def gen_goals(wall_x, wall_z,wall_size, diff, num=3, side='left'):
    """Num is num of goals on each side of wall"""
    res = []
    if 'left':
        sign = -1
    else:
        sign = 1
    for i in range(0,num):
        specs = {
            'x': wall_x+(sign*((wall_size/2)+0.5+i))-sign*max(0,min(diff-1,3)),
            'z': wall_z+min(diff,3)
        }
        res.append(a_goal.format(**specs))
    return "".join(res)

num_arenas = 50
curric_num = 8
for i in range(0,num_arenas):
    
    locs = valid_loc(int(i/10), side='right')
    if any((v>39)|(v<0) for k,v in locs.items() if k not in ['agent_rot','goals']):
        raise Exception(f"BAD ENTRY {locs}")
    arena = """!ArenaConfig
    arenas:
      -1: !Arena
        items:
        - !Item
          name: Agent
          positions:
          - !Vector3 {{x: {agent_x}, y: 0, z: {agent_z}}}
          rotations: [{agent_rot}]{goals}
        - !Item
          name: Wall
          positions:
          - !Vector3 {{x: {wall_x}, y: 0, z: {wall_z}}}
          sizes:
          - !Vector3 {{x: {wall_size}, y: {wall_tall}, z: 2}}
          rotations: [0]
          colors: 
          - !RGB {{r: 153, g: 153, b: 153}}
        pass_mark: 0
        t: 250""".format(**locs)
    text_file = open(f"configurations/curriculum{curric_num}/{i}.yml", "w")
    text_file.write(arena)
    text_file.close()
# for i in range(num_arenas,num_arenas*2):
    
#     locs = valid_loc(int(i/10))
#     if any((v>39)|(v<0) for k,v in locs.items() if k not in ['agent_rot','goals']):
#         raise Exception(f"BAD ENTRY {locs}")
#     arena = """!ArenaConfig
#     arenas:
#       -1: !Arena
#         items:
#         - !Item
#           name: Agent
#           positions:
#           - !Vector3 {{x: {agent_x}, y: 0, z: {agent_z}}}
#           rotations: [{agent_rot}]{goals}
#         - !Item
#           name: Wall
#           positions:
#           - !Vector3 {{x: {wall_x}, y: 0, z: {wall_z}}}
#           sizes:
#           - !Vector3 {{x: {wall_size}, y: {wall_tall}, z: 2}}
#           rotations: [0]
#           colors: 
#           - !RGB {{r: 153, g: 153, b: 153}}
#         pass_mark: 0
#         t: 250""".format(**locs)
#     text_file = open(f"configurations/curriculum{curric_num}/{i}.yml", "w")
#     text_file.write(arena)
#     text_file.close()


In [51]:
a_nums = []
for i in range(num_arenas):
    a_nums.append(f"{i}.yml")
a_thresh = []
for i in range(num_arenas-1):
    level = round(0.8-0.05*int(i/10),2)
    level = -1
    a_thresh.append(level)
json = {
    "measure":"reward",
    "min_lesson_length":1,
    "signal_smoothing":True,
    "thresholds":a_thresh,
    "configuration_files":a_nums
}
text_file = open(f"configurations/curriculum{curric_num}/AnimalAI.json", "w")
text_file.write(str(json).replace("'", '"').replace(",", ",\n").replace('True','true'))
text_file.close()


In [27]:
randrange(0,1)

0